In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [2]:
df = pd.read_csv('/content/hotstar.csv')
df.head

<bound method NDFrame.head of       hotstar_id                               title  \
0     1000232077              Ram Gopal Varma ki Aag   
1     1000164652                             Niyogam   
2     1000105823            Injakkadan Mathai & Sons   
3     1000163530                    Viyarppinte Vila   
4     1820041764                          Ek Mulaqat   
...          ...                                 ...   
6647  1260003841                         Anandyatree   
6648  1260108360                    Cook Like a Chef   
6649  1260067695  Star Kids - Daddy Daddy O My Daddy   
6650  1260025773                     Disney's Recess   
6651  1260001799      Ind vs WI 2018 Countdown Tamil   

                                            description         genre  year  \
0     Dacoit Babban Singh is on the loose in Kaligan...        Action  2007   
1     A mediator between the deity and the devotees,...         Drama  1997   
2     Injakkadan Mathai and his sons, Roy and Thanka...     

In [3]:
df = df.drop(columns=['title'])
df = df.drop(columns=['hotstar_id'])
df = df.drop(columns=['year'])
df = df.drop(columns=['age_rating'])
df = df.drop(columns=['running_time'])
df = df.drop(columns=['seasons'])
df = df.drop(columns=['episodes'])
df = df.drop(columns=['genre'])
df.head

<bound method NDFrame.head of                                             description   type
0     Dacoit Babban Singh is on the loose in Kaligan...  movie
1     A mediator between the deity and the devotees,...  movie
2     Injakkadan Mathai and his sons, Roy and Thanka...  movie
3     An impoverished landlord lives in past glory a...  movie
4     Two strangers, Avi and Nikita meet at a party ...  movie
...                                                 ...    ...
6647  Popular Marathi singers, Suresh Wadkar, Swapni...     tv
6648  This innovative, ground-breaking series featur...     tv
6649  Vijay TV stars and their adorable tiny tots ar...     tv
6650  Recess portrays the lives of six fourth-grader...     tv
6651  Watch the best moments from West Indies' tour ...     tv

[6652 rows x 2 columns]>

In [4]:
df.dropna(axis=0, subset=['description', 'type'], inplace=True)
category = pd.get_dummies(df.type)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='type')
df_baru

,description,movie,tv
0,Dacoit Babban Singh is on the loose in Kaligan...,1,0
1,"A mediator between the deity and the devotees,...",1,0
2,"Injakkadan Mathai and his sons, Roy and Thanka...",1,0
3,An impoverished landlord lives in past glory a...,1,0
4,"Two strangers, Avi and Nikita meet at a party ...",1,0
...,...,...,...
6647,"Popular Marathi singers, Suresh Wadkar, Swapni...",0,1
6648,"This innovative, ground-breaking series featur...",0,1
6649,Vijay TV stars and their adorable tiny tots ar...,0,1
6650,Recess portrays the lives of six fourth-grader...,0,1


In [5]:
deskripsi = df_baru['description'].values
label = df_baru[['movie', 'tv']].values

In [6]:
deskripsi_latih, deskripsi_test, label_latih, label_test = train_test_split(deskripsi, label, test_size=0.2)

In [7]:
tokenizer = Tokenizer(num_words=5000, oov_token='<oov>', filters ='!"#$%()*+,-./:;<=>?@[\]^_`{|}~', lower=True) 
tokenizer.fit_on_texts(deskripsi_latih) 
tokenizer.fit_on_texts(deskripsi_test)
 
sekuens_latih = tokenizer.texts_to_sequences(deskripsi_latih)
sekuens_test = tokenizer.texts_to_sequences(deskripsi_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [9]:
class myCallback(tf.keras.callbacks.Callback):

  def on_epoch_end(self, epoch, logs={}):

    if(logs.get('accuracy') > 0.75  and logs.get('val_accuracy') > 0.75 ):  

      print("\nAkurasi dan validasi telah mencapai >75%!")

      self.model.stop_training = True

callbacks = myCallback()

In [10]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2, callbacks=[callbacks])

Epoch 1/30
167/167 - 20s - loss: 0.5524 - accuracy: 0.7247 - val_loss: 0.4395 - val_accuracy: 0.7956 - 20s/epoch - 119ms/step
Epoch 2/30

Akurasi dan validasi telah mencapai >75%!
167/167 - 10s - loss: 0.3358 - accuracy: 0.8581 - val_loss: 0.5154 - val_accuracy: 0.7941 - 10s/epoch - 62ms/step
